# **Operations to be done with google drive**

In [ ]:
#@title Linking with Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Prerequisite Operations**

In [ ]:
#@title Installing Dependencies

%pip install tqdm
%pip install -U segmentation-models
%pip install focal-loss

In [ ]:
#@title Requirements

import tensorflow as tf
import os

In [ ]:
#@title Choosing Resources

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
   tf.config.experimental.set_memory_growth(gpu, True)

# **Data Processing**

In [ ]:
#@title Data Cleansing

import cv2
import imghdr
from matplotlib import pyplot as plt
import random

data_dir = ['/kaggle/input/3dircadb-hu-clipped']
os.mkdir(os.path.join('/kaggle/working/','logs'))
os.mkdir(os.path.join('/kaggle/working/','models'))

image_exts = ['jpg', 'jpeg', 'png', 'bmp']

for i in data_dir:
    for image_class in os.listdir(i):
        for image in os.listdir(os.path.join(i, image_class)):
            image_path = os.path.join(i, image_class, image)
            try:
                img0 = cv2.imread(image_path)
                tip = imghdr.what(image_path)
                if tip not in image_exts:
                    print('Image not in ext list {}'.format(image_path))
                    os.remove(image_path)

            except Exception as e:
                print('Issue with image {}'.format(image_path))

In [ ]:
#@title Creating Data Pipeline and Data Preprocessing
import numpy as np
from PIL import Image
from glob import glob
from sklearn.model_selection import train_test_split

image_directory = '/kaggle/input/3dircadb-hu-clipped/CT_Images'
mask_directory = '/kaggle/input/3dircadb-hu-clipped/Liver_masks'

SIZE = 256
batch_size = 8
image_dataset = []
mask_dataset = []

def load_dataset(path, split=0.2):
    images = sorted(glob(os.path.join(path, "CT_Images", "*.jpg")))
    masks = sorted(glob(os.path.join(path, "Liver_masks", "*.jpg")))

    split_size = int(len(images) * split)

    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=split_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=split_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (SIZE, SIZE))
    x = x / 255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (SIZE, SIZE))
    x = x / 255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_image(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([SIZE, SIZE, 3])
    y.set_shape([SIZE, SIZE, 3])
    return x, y

def tf_dataset(X, Y, batch=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset


(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(data_dir[0])

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")
print(f"Test : {len(test_x)} - {len(test_y)}")

train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

# **Deep Learning Models Creation, Training, Evaluating, Saving, Performance Graph etc.**

In [ ]:
#@title Score Calculations

import keras.backend as K

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)


# Focal Loss
from focal_loss import BinaryFocalLoss

def focal_loss(y_true, y_pred):
    loss_func = BinaryFocalLoss(gamma=2)
    loss = loss_func(y_true, y_pred)
    return loss


# Hybrid Loss
def hybrid_loss(y_true, y_pred):
    dice = dice_loss(y_true, y_pred)
    focal = focal_loss(y_true, y_pred)
    total_loss = dice+(2*focal)
    return total_loss


# Intersection Over Union (IOU) calculation
def mean_iou(y_true, y_pred):
    y_pred = tf.round(tf.cast(y_pred, tf.int32))
    intersect = tf.reduce_sum(tf.cast(y_true, tf.float32) * tf.cast(y_pred, tf.float32), axis=[1])
    union = tf.reduce_sum(tf.cast(y_true, tf.float32),axis=[1]) + tf.reduce_sum(tf.cast(y_pred, tf.float32),axis=[1])
    smooth = tf.ones(tf.shape(intersect))
    return tf.reduce_mean((intersect + smooth) / (union - intersect + smooth))

def iou(y_true, y_pred):
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    iou_score = intersection / (union + smooth)
    return iou_score

def iou_loss(y_true, y_pred):
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(tf.cast(y_true, tf.float32) * tf.cast(y_pred, tf.float32))
    score = (intersection + 1.) / (tf.reduce_sum(tf.cast(y_true, tf.float32)) +
    tf.reduce_sum(tf.cast(y_pred, tf.float32)) - intersection + 1.)
    return 1 - score

def batch_iou_loss(y_true, y_pred):
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(tf.cast(y_true, tf.float32) * tf.cast(y_pred, tf.float32))
    union = tf.reduce_sum(tf.cast(y_true, tf.float32)) + tf.reduce_sum(tf.cast(y_pred, tf.float32)) - intersection
    iou_score = (intersection + 1.) / (union + 1.)
    iou_loss = 1 - iou_score
    return iou_loss

In [ ]:
#@title Deep Models

''' Deep Model '''
# The Network
import keras.backend as K
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Layer, Conv2D, MaxPool2D, Dense, Flatten, Dropout, AveragePooling2D, Concatenate, Conv2DTranspose, GlobalAveragePooling2D, BatchNormalization, ReLU, Reshape, Resizing, Add, SeparableConv2D, MultiHeadAttention, UpSampling2D, LayerNormalization
from tensorflow.keras.applications import Xception, MobileNetV2, DenseNet201
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import Recall, Precision, MeanIoU, Accuracy

import os
os.environ["SM_FRAMEWORK"] = "tf.keras"
from tensorflow import keras
import segmentation_models as sm

def unet(img_shape, n_classes):
    model = sm.Unet('mobilenetv2', input_shape=img_shape, encoder_weights='imagenet', classes=n_classes+1, activation='sigmoid')

    return model


def linknet(img_shape, n_classes):
    model = sm.Linknet('mobilenetv2', input_shape=img_shape, encoder_weights='imagenet', classes=n_classes+1, activation='sigmoid')

    return model


def fpn(img_shape, n_classes):
    model = sm.FPN('mobilenetv2', input_shape=img_shape, encoder_weights='imagenet', classes=n_classes+1, activation='sigmoid')

    return model


input_shape = (256, 256, 3)
n_classes = 2


model1 = unet(input_shape, n_classes)
model1.compile('Adam', loss=hybrid_loss, metrics=['accuracy', 'Precision', 'Recall', dice_coef, mean_iou])
model1.summary()

model2 = linknet(input_shape, n_classes)
model2.compile('Adam', loss=hybrid_loss, metrics=['accuracy', 'Precision', 'Recall', dice_coef, mean_iou])
model2.summary()

model3 = fpn(input_shape, n_classes)
model3.compile('Adam', loss=hybrid_loss, metrics=['accuracy', 'Precision', 'Recall', dice_coef, mean_iou])
model3.summary()

In [ ]:
#@title Training the Models

# Train
logdir='/kaggle/working/logs'
tensorboard_callback1 = tf.keras.callbacks.TensorBoard(log_dir=logdir)
tensorboard_callback2 = tf.keras.callbacks.EarlyStopping(monitor="val_dice_coef", min_delta=0.0010, patience=5, verbose=1, mode='max', restore_best_weights=True, start_from_epoch=0)
hist1 = model1.fit(train_dataset, epochs=60, validation_data=valid_dataset, callbacks=[tensorboard_callback1, tensorboard_callback2])
print(hist1.history)

tensorboard_callback1 = tf.keras.callbacks.TensorBoard(log_dir=logdir)
tensorboard_callback2 = tf.keras.callbacks.EarlyStopping(monitor="val_dice_coef", min_delta=0.0010, patience=5, verbose=1, mode='max', restore_best_weights=True, start_from_epoch=0)
hist2 = model2.fit(train_dataset, epochs=60, validation_data=valid_dataset, callbacks=[tensorboard_callback1, tensorboard_callback2])
print(hist2.history)

tensorboard_callback1 = tf.keras.callbacks.TensorBoard(log_dir=logdir)
tensorboard_callback2 = tf.keras.callbacks.EarlyStopping(monitor="val_dice_coef", min_delta=0.0010, patience=5, verbose=1, mode='max', restore_best_weights=True, start_from_epoch=0)
hist3 = model3.fit(train_dataset, epochs=60, validation_data=valid_dataset, callbacks=[tensorboard_callback1, tensorboard_callback2])
print(hist3.history)

In [ ]:
#@title Evaluating the Models

test_dataset = tf_dataset(test_x, test_y, batch=32)

test_steps = (len(test_x)//32)
if len(test_x) % 32 != 0:
    test_steps += 1

print("For Unet : ")
model1.evaluate(test_dataset, steps=test_steps)

print("For Linknet : ")
model2.evaluate(test_dataset, steps=test_steps)

print("For FPN : ")
model3.evaluate(test_dataset, steps=test_steps)

In [ ]:
#@title Testing the Models

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = cv2.resize(x, (256, 256))
    x = x/255.0
    return x

In [ ]:
#@title Performance Graphs

import matplotlib.pyplot as plt

''' Unet '''
# Loss
fig = plt.figure()
plt.title("Unet\n\n", loc='center', fontsize=26)
plt.plot(hist1.history['loss'], color='teal', label='dice_loss')
plt.plot(hist1.history['val_loss'], color='orange', label='val_dice_loss')
fig.suptitle('Dice Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# Dice Coefficient
fig = plt.figure()
plt.plot(hist1.history['dice_coef'], color='teal', label='dice_coef')
plt.plot(hist1.history['val_dice_coef'], color='orange', label='val_dice_coef')
fig.suptitle('Dice Coefficient', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# Intersection Over Union (IOU)
fig = plt.figure()
plt.plot(hist1.history['mean_iou'], color='teal', label='mean_iou')
plt.plot(hist1.history['val_mean_iou'], color='orange', label='val_mean_iou')
fig.suptitle('Intersection Over Union (IOU)', fontsize=20)
plt.legend(loc="upper left")
plt.show()


''' Linknet '''
# Loss
fig = plt.figure()
plt.title("Linknet\n\n", loc='center', fontsize=26)
plt.plot(hist2.history['loss'], color='teal', label='dice_loss')
plt.plot(hist2.history['val_loss'], color='orange', label='val_dice_loss')
fig.suptitle('Dice Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# Dice Coefficient
fig = plt.figure()
plt.plot(hist2.history['dice_coef'], color='teal', label='dice_coef')
plt.plot(hist2.history['val_dice_coef'], color='orange', label='val_dice_coef')
fig.suptitle('Dice Coefficient', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# Intersection Over Union (IOU)
fig = plt.figure()
plt.plot(hist2.history['mean_iou'], color='teal', label='mean_iou')
plt.plot(hist2.history['val_mean_iou'], color='orange', label='val_mean_iou')
fig.suptitle('Intersection Over Union (IOU)', fontsize=20)
plt.legend(loc="upper left")
plt.show()


''' FPN '''
# Loss
fig = plt.figure()
plt.title("FPN\n\n", loc='center', fontsize=26)
plt.plot(hist3.history['loss'], color='teal', label='dice_loss')
plt.plot(hist3.history['val_loss'], color='orange', label='val_dice_loss')
fig.suptitle('Dice Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# Dice Coefficient
fig = plt.figure()
plt.plot(hist3.history['dice_coef'], color='teal', label='dice_coef')
plt.plot(hist3.history['val_dice_coef'], color='orange', label='val_dice_coef')
fig.suptitle('Dice Coefficient', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# Intersection Over Union (IOU)
fig = plt.figure()
plt.plot(hist3.history['mean_iou'], color='teal', label='mean_iou')
plt.plot(hist3.history['val_mean_iou'], color='orange', label='val_mean_iou')
fig.suptitle('Intersection Over Union (IOU)', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
#@title Saving the Models and their Histories

import pickle


# Saving the Models

model1.save(os.path.join('/kaggle/working/models','unet.h5'))
model2.save(os.path.join('/kaggle/working/models','linknet.h5'))
model3.save(os.path.join('/kaggle/working/models','fpn.h5'))


# Saving the Model Histories

with open(os.path.join(logdir+'/unet.pkl'), 'wb') as file:
    pickle.dump(hist1, file)

with open(os.path.join(logdir+'/linkunet.pkl'), 'wb') as file:
    pickle.dump(hist2, file)

with open(os.path.join(logdir+'/fpn.pkl'), 'wb') as file:
    pickle.dump(hist3, file)

In [ ]:
#@title Extracting Results of each Model

from tqdm import tqdm

p1 = []; p2 = []; p3 = []; labels = []
for i, (x, y) in tqdm(enumerate(zip(test_x, test_y))):
    x = read_image(x)
    y = read_image(y)
    y_pred1 = model1.predict(np.expand_dims(x, axis=0), verbose=0)
    y_pred2 = model2.predict(np.expand_dims(x, axis=0), verbose=0)
    y_pred3 = model3.predict(np.expand_dims(x, axis=0), verbose=0)
    labels.append(y)
    p1.append(y_pred1)
    p2.append(y_pred2)
    p3.append(y_pred3)

labels = np.asarray(labels)
p1 = np.asarray(p1)
p2 = np.asarray(p2)
p3 = np.asarray(p3)

In [ ]:
#@title Blumberg Function

def fuzzy_rank(CF, top):
    R_L = np.zeros(CF.shape)
    for h in range(CF.shape[0]):
        for i in range(CF.shape[1]):
            for j in range(CF.shape[2]):
                for k in range(CF.shape[3]):
                    for m in range(CF.shape[4]):
                        for n in range(CF.shape[5]):
                            for p in range(CF.shape[6]):
                                R_L[h, i, j, k, m, n, p] = 1 - (1 * (CF[h, i, j, k, m, n, p]) ** 1) / (0.5 + (CF[h, i, j, k, m, n, p]) ** 1)

    K_L = 1*np.ones(shape = R_L.shape)
    for i in range(R_L[0].shape[0]):
        for j in range(R_L[0].shape[1]):
            for k in range(R_L[0].shape[2]):
                for m in range(R_L[0].shape[3]):
                    for n in range(R_L[0].shape[4]):
                        for p in range(R_L[0].shape[5]):
                            if R_L[0, i, j, k, m, n, p] < R_L[1, i, j, k, m, n, p]:
                                K_L[0, i, j, k, m, n, p] = R_L[0, i, j, k, m, n, p]
                            else:
                                K_L[1, i, j, k, m, n, p] = R_L[1, i, j, k, m, n, p]

    return K_L


def CFS_func(CF, K_L):
    H = CF.shape[1]
    for i in range(K_L[0].shape[0]):
        for j in range(K_L[0].shape[1]):
            for k in range(K_L[0].shape[2]):
                for m in range(K_L[0].shape[3]):
                    for n in range(K_L[0].shape[4]):
                        for p in range(K_L[0].shape[5]):
                            if K_L[0, i, j, k, m, n, p] == 1:
                                CF[0, i, j, k, m, n, p] = 0
                            elif K_L[1, i, j, k, m, n, p] == 1:
                                CF[1, i, j, k, m, n, p] = 0
    CFR = [np.sum(CF[0], axis=0), np.sum(CF[1], axis=0)]
    CFR = np.asarray(CFR)
    CFS = 1 - CFR/H
    return CFS


def Blumberg(top = 2, *argv):
    L = len(argv)

    arg_shape = argv[0].shape
    CF = np.zeros(shape=(2, L,) + arg_shape)

    for i, arg in enumerate(argv):
        CF[0][i] = arg
        CF[1][i] = 1 - arg

    R_L = fuzzy_rank(CF, top)
    RS = [np.sum(R_L[0], axis=0), np.sum(R_L[1], axis=0)]

    RS = np.asarray(RS)
    CFS = CFS_func(CF, R_L)
    FS = RS*CFS

    predictions = np.zeros(FS[0].shape)
    for i in range(FS[0].shape[0]):
        for j in range(FS[0].shape[1]):
            for k in range(FS[0].shape[2]):
                for m in range(FS[0].shape[3]):
                    for n in range(FS[0].shape[4]):
                        if FS[0, i, j, k, m, n] < FS[1, i, j, k, m, n]:
                            predictions[i, j, k, m, n] = 1
                        else:
                            predictions[i, j, k, m, n] = 0

    return predictions

predictions = Blumberg(2, p1, p2, p3)

In [ ]:
#@title Blumberg Results

dice = dice_coef(labels, predictions)

print("Ensemble Dice Coefficient : ", dice)

iou = mean_iou(labels, predictions)

print("Ensemble IOU : ", iou)

Ensemble Dice Coefficient :  tf.Tensor(0.9773762, shape=(), dtype=float32)
